In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))
print(f"Length Train: {len(train_df)}")
print(f"Length Test: {len(test_df)}")
# train_df.head()

In [ ]:
X_train, y_train = train_df.drop(['Severity', 'sequence_id'], axis=1), train_df['Severity']
X_test, y_test = test_df.drop(['Severity', 'sequence_id'], axis=1), test_df['Severity']

In [ ]:
X_train.head()

In [ ]:
def modify_X(df):
    result = df.copy()
    result = result.to_numpy()
    result = result.reshape(-1, 21350//25, 50)
    return result
X_train_modified = modify_X(X_train)
X_train_modified.shape

In [ ]:
def modify_y(df):
    result = pd.get_dummies(df)
    result = result.to_numpy()
    return result
y_train_modified = modify_y(y_train)
y_train_modified.shape

In [ ]:
def create_model(input_shape, n_output):
    model = Sequential([
        InputLayer(input_shape),
        Bidirectional(LSTM(100, return_sequences=True)),
        Bidirectional(LSTM(50, return_sequences=True)),
        Bidirectional(LSTM(50, return_sequences=False)),
        Dense(n_output, activation="softmax")
    ])
    return model
model = create_model((X_train_modified.shape[1], X_train_modified.shape[2]),
                     y_train_modified.shape[1])
model.summary()

In [ ]:
optimizer = Adam(learning_rate=1e-3)
loss = CategoricalCrossentropy()
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.3,
                              patience=5, min_lr=1e-5)

In [ ]:
history = model.fit(X_train_modified, y_train_modified,
          batch_size=32,
          epochs=200,
          shuffle=True,
          validation_split=0.2,
          callbacks=[reduce_lr])

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validate'], loc='upper left')
plt.show()